# TP 2 : Régression logistique

## 1 Régularisation de Tikhonov

### 1.1 : Étude de $f_1$

**Calcul du gradient :**

$\frac{df_1}{dw_0} (w_0, w)= \frac 1 n \sum_{i=1}^n{\frac{-y_i}{1+exp(y_i(x_i^T*w + w_0))}}$


$\frac{df_1}{dw} (w_0, w)= \frac 1 n \sum_{i=1}^n{\frac{-y_i*x_i}{1+exp(y_i(x_i^T*w + w_0))}} + \rho * w$

**Calcul de la hessienne :**

$H = \frac 1 n * \sum_i{\frac{y_i^2}{(2cosh(y_i/2(x_i^T*w + w_0))^2}} * \begin{bmatrix}1 \\ x_i \end{bmatrix} \begin{bmatrix}1 \\ x_i \end{bmatrix}^T + \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix}  $

Montrons que $H$ est définie positive. Posons $a_i = \frac{y_i}{\sqrt n (2cosh(y_i/2(x_i^T*w + w_0))}$ et $v_i = \begin{bmatrix}1 \\ x_i \end{bmatrix}$, on a :

$\forall u \in \mathbb R^{p+1}, u^T H u = u^T \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix} u + \sum_i{(u^T a_i v_i) (v_i^T a_i u) }$

* $u^T \begin{vmatrix} 0 & 0 \\ 0 & {\rho * I_p} \end{vmatrix} u = \begin{bmatrix} 0 & {\rho \sum_{i=2}^{p+1}{u_i}} & {...} & {\rho \sum_{i=2}^{p+1}{u_i}} \end{bmatrix} \begin{bmatrix} u_1 \\ ... \\ u_ {p+1} \end{bmatrix} = \rho (\sum_{i=2}^{p+1}{u_i})^2 > 0$
* $\sum_i{(u^T a_i v_i) (v_i^T a_i u) } = \sum_i{(u^T a_i v_i)^2 } > 0$

Ainsi $ u^T H u > 0$. 

Et donc $f_1$ est convexe